In [ ]:
# Import all required libraries
# Read csv the dataframe
# Inspect the dataframe and clean if required
# Make new dataframe with selected feature columns
# Scale the features
# Test train split (Randon seed, shuffle, stratify)
# Create KNN model and fit the training data
# Predict the test data
# Evaluate the model using recall and precision. Present a confusion matrix.
# Perform cross validation with 5 folds and report average recall and precision
# Perform hyperparameter tuning using GridSearchCV for n_neighbors from 1 to 20
# Create a pipeline including scaling and KNN model
# Add visualisations


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [6]:
ttc_initial=pd.read_csv('https://github.com/shahab-malik748/ml13-ttc-delays-project/blob/arman/src_files/ttc_delays_final_df.csv') 

ParserError: Error tokenizing data. C error: Expected 1 fields in line 38, saw 2


In [ ]:
# label target variable major_delay_flag
ttc_initial["major_delay_flag"] = ttc_initial["major_delay_flag"].replace({
    "0" : "no_major_delay",
    "1" : "major_delay",
})

In [ ]:
# Select features and target variable and create new dataframe
# month	day_of_month	hour	day_of_week	delay_category	line	major_delay_flag

ttc=ttc_initial[['month','day_of_month','hour','day_of_week','line','major_delay_flag','delay_category']]

In [ ]:
# Feature Scaling

# Create a copy of the original 'ttc' dataframe to ensure we're not modifying the original data
standardized_ttc = ttc.copy()

# Specify the columns that we do NOT want to scale (ID and diagnosis columns)
columns_to_exclude = ['major_delay_flag']

# Select the columns that we want to scale by excluding the 'id' and 'diagnosis' columns
# This will return a list of the numeric columns we need to scale
columns_to_scale = standardized_ttc.columns.difference(columns_to_exclude)

# Initialize the StandardScaler to standardize the selected numeric columns
scaler = StandardScaler()

# Apply the scaler to the selected columns. This transforms the data so that each feature
# has a mean of 0 and a standard deviation of 1, which is essential to prevent larger
# scale features from dominating the analysis, especially for distance-based algorithms like KNN.
standardized_ttc[columns_to_scale] = scaler.fit_transform(ttc[columns_to_scale])

# Output the standardized dataframe with the scaled numeric columns
standardized_ttc
